# Exportar cartografías iterando rásters

Este script nace en la necesidad de poder generar cartografías por lote utilizando ArcGIS Pro por sobre otros softwares o herramientas. ArcGIS Pro ofrece una calidad cartográfica superior a R, que es otra herramienta para poder exportar cartografías en lotes.

El objetivo es cargar un archivo ráster, extraer el nombre, aplicar una simbología, cambiar el título de la cartografía y exportar la imagen. Cada una de estas tendra por objetivo generar un gif animado de la cobertura de nieve en los últimos años. Crear +1.000 cartografías manualmente no es una opción.

El procedimiento es el siguiente. Como siempre, lo primero es cargar las librerías de trabajo:

In [1]:
# Cargando arcpy
import arcpy
# Para listar como diccionario
import pprint
# Librerías para acceder al sistema
import os
import sys
# Para separar año y día del año
import re
# Para convertir fechas
from datetime import datetime

En la siguiente línea se puede colocar la ruta del proyecto donde está el layout que se quiere iterar. Pare este caso es `"CURRENT"` debido a que estamos corriendo la rutina desde el proyecto cargado en ArcGIS Pro.

In [2]:
# Listar proyecto actual
aprx = arcpy.mp.ArcGISProject("CURRENT")

Luego se llaman los elementos necesarios para poder iterar:
 - `nieve_map`: mapa llamado Nieve que es el seleccionado para iterar los rásters sobre él.
 - `nieve`: ráster ya cargado de nombre `ndsi_terra_cqbo_%Y-%j.tif`, este es el archivo que se pretende iterar.
 - `symbology`: objeto con la simbología de la capa antes mencionada, para que todas las iteraciones tengan la misma simbología.
 - `rasters`: listado de los nombres de los rásters que se iterarán.
 - `lyt`: Composición de diseño que se exportará
 - `main`: Cuadro de texto que contiene el título de la cartografía, se cambiará en cada iteración.
 - `group`: grupo de capas creado para posicionar la capa cargada entre ciertas coberturas críticas dejando los límites politico-administrativos, la red hidrográfica y ciudades por sobre la cobertura de nieve.

In [3]:
# LLamar los elementos a modificar
nieve_map = aprx.listMaps('Nieve')[0]
nieve = nieve_map.listLayers('ndsi*')[0]
symbology = nieve.symbology
rasters = os.listdir(r"C:\Users\Aldo Tapia\Desktop\span0015b")
lyt = aprx.listLayouts('Nieve')[0]
main = lyt.listElements('TEXT_ELEMENT','Titulo')[0]
group = nieve_map.listLayers('Nieve')[0]

El bucle `for` es sencillo. Se carga la ruta el archivo a iterar y del nombre se extrae la fecha para ponerla en dos tipos de formatos, uno para el título y el otro para el nombre del archivo. Luego se elimina el ráster que está en el mapa, se carga el archivo a iterar y se mueve al grupo previamente definido. Finalmente se exporta la composición en formato PNG.

In [4]:
for x in rasters:
    path = os.path.join(r"C:\Users\Aldo Tapia\Desktop\span0015b",x)
    date = re.findall(r'\d+',x)
    year = date[0]
    doy = date[1]
    date2 = datetime.strptime(year + "-" + doy, "%Y-%j").strftime("%d-%m-%Y")
    fdate = datetime.strptime(year + "-" + doy, "%Y-%j").strftime("%Y-%m-%d")
    fname = fdate + '.png'
    main.text = 'Cobertura nival de \n' + date2
    nieve_map.removeLayer(nieve)
    nieve_map.addDataFromPath(path)
    nieve = nieve_map.listLayers('ndsi*')[0]
    nieve.symbology = symbology
    nieve_map.addLayerToGroup(group,nieve)
    nieve_map.removeLayer(nieve)
    nieve = nieve_map.listLayers('ndsi*')[0]
    lyt.exportToPNG(os.path.join(r'C:\Users\Aldo Tapia\Desktop\cartos_nieve',fname),300)